In [2]:
#to use a local file in google collab
from google.colab import files 

#upload a file to collab using the browse button
uploaded = files.upload()

Saving Gear25.csv to Gear25.csv


In [0]:
import io 
import pandas as pd 
import numpy as np 

#import libraries to implement different classifiers
#import library to split test and train data
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


#libraries for authenticating a user to link a google account to access spreadhsheets
#then importing google spreadsheets
from google.colab import auth 
from oauth2client.client import GoogleCredentials
import gspread  

In [4]:
#authenticate a user 
#user needs to go to the link and select a google account to be linked
#then the user needs to give permission for access
#once permission is granted google speadsheet will be linked with the authenticated account
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

#create a google spreadsheet with this name "G25_out"
sh = gc.create('G25_out.csv')

#decode and read the dataset
df = pd.read_csv(io.StringIO(uploaded['Gear25.csv'].decode('utf-8'))) 

#to remove columns with 0's
df = df.loc[: , (df!= 0).any(axis = 0)] 

#Takes all rows of all columns except the last column
X = df.iloc[:,:-1].values

#Takes all rows of the last column
Y = df.iloc[:,-1].values 

#open the google spreadsheet
#the output of this code will be updated to this google spreadsheet
#this spreadsheet can be accessed from google drive
worksheet = gc.open('G25_out.csv').sheet1 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
#our aim is to get the output from fold no#1 to no#10
#loop variable is used to change the cross_validation values from 1 to 10
loop = 1
for x in range (1,11): 
  if loop >= 2: 
    
    #to assign the loop value to variable x
    #this variable x is the crossvalidation value
    cv = x
    
    #splitting data into test and train
    #test data is 20%
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2) 
    
    #Fit to data, then transform it
    #Perform standardization by centering and scaling
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train) 
    X_test = sc_X.transform(X_test)
     
    #prints the number of folds
    print("for CV value =",cv,"we have the following output:\n")
    
    #for RandomForest classifier and tuning hyperparameter
    clf = RandomForestClassifier(n_estimators=100) 
    np.mean(cross_val_score(clf, X_train, Y_train, cv=cv))
    clf.fit(X_train,Y_train)
    clf_predict = clf.predict(X_test) 
    
    #for RBF classifier and tuning hyperparameter
    svclassifier = SVC(kernel='rbf', gamma='scale', C= 1000)
    np.mean(cross_val_score(svclassifier, X_train, Y_train, cv=cv))
    svclassifier.fit(X_train, Y_train) 
    svc_predictions = svclassifier.predict(X_test) 
    
    #codes works fine but takes too much time to process
    #to test output uncomment this from here...
    #for Multi Layer Perceptron classifier and tuning hyperparameter
    #mlp = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=1200)  
    #np.mean(cross_val_score(mlp, X_train, Y_train, cv=cv)) 
    #mlp.fit(X_train, Y_train) 
    #mlp_predictions = mlp.predict(X_test)
    
    #for Decision tree classifier
    DT = DecisionTreeClassifier()
    np.mean(cross_val_score(svclassifier, X_train, Y_train, cv=cv))
    DT.fit(X_train,Y_train)
    DT_predictions = DT.predict(X_test)
    
    #for k-NN classifier with neighbors = 1 
    knn = KNeighborsClassifier(n_neighbors=1) 
    np.mean(cross_val_score(knn, X_train, Y_train, cv=cv)) 
    knn.fit(X_train, Y_train) 
    knn_predictions = knn.predict(X_test)
    
    #print the output
    #update the google sheet with the outputs
    #for RF
    print('RF F1score:', f1_score(Y_test, clf_predict, average= 'weighted'))
    cell_list = worksheet.update_cell(cv + 1, 1, f1_score(Y_test, clf_predict, average= 'weighted'))
    print('RF Accuracy:', accuracy_score(Y_test, clf_predict))
    cell_list = worksheet.update_cell(cv + 1, 2, accuracy_score(Y_test, clf_predict))
    
    #for RBF
    print('RBF F1score:', f1_score(Y_test, svc_predictions, average= 'weighted'))
    cell_list = worksheet.update_cell(cv + 1, 3, f1_score(Y_test, svc_predictions, average= 'weighted'))                                  
    print('RBF Accuracy:', accuracy_score(Y_test, svc_predictions))
    cell_list = worksheet.update_cell(cv + 1, 4, accuracy_score(Y_test, svc_predictions))  
    
    #uncomment to check f1 score of MLP
    #results for MLP have not been linked to google spreadsheet
    #print('MLP F1score:', f1_score(Y_test, mlp_predictions, average= 'weighted'))
    #uncomment to check accuracy of MLP
    #results for MLP have not been linked to google spreadsheet
    #print('MLP Accuracy:', accuracy_score(Y_test, mlp_predictions))
    
    #for DT
    print('DT F1score:', accuracy_score(Y_test, DT_predictions))
    cell_list = worksheet.update_cell(cv + 1, 5, f1_score(Y_test, DT_predictions, average= 'weighted'))                                       
    print('DT Accuracy:', accuracy_score(Y_test, DT_predictions))
    cell_list = worksheet.update_cell(cv + 1, 6, accuracy_score(Y_test, DT_predictions))
    
    #for k-NN
    print('kNN F1score:', f1_score(Y_test, knn_predictions, average= 'weighted'))
    cell_list = worksheet.update_cell(cv + 1, 7, f1_score(Y_test, knn_predictions, average= 'weighted'))                                  
    print('kNN F1score:', accuracy_score(Y_test, knn_predictions))
    cell_list = worksheet.update_cell(cv + 1, 8, accuracy_score(Y_test, knn_predictions))
    print("\n")

  if loop < 2: 
    
    loop = loop +1 
    print("for CV value =",1,"we have the following output:\n")
    
    #splitting data into test and train
    #test data is 20%
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2) 
    
    #Fit to data, then transform it
    #Perform standardization by centering and scaling
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train) 
    X_test = sc_X.transform(X_test)

    #for RandomForest classifier and tuning hyperparameter
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,Y_train)
    clf_predict = clf.predict(X_test)
    
    #for RBF classifier and tuning hyperparameter
    svclassifier = SVC(kernel='rbf',gamma='scale', C= 1000)
    svclassifier.fit(X_train, Y_train)
    svc_predictions = svclassifier.predict(X_test)
    
    #codes works fine but takes too much time to process
    #to test output uncomment this from here...
    #for Multi Layer Perceptron classifier and tuning hyperparameter
    #mlp = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=1200) 
    #mlp.fit(X_train, Y_train)
    #mlp_predictions = mlp.predict(X_test)
    
    #for Decision tree classifier
    DT = DecisionTreeClassifier()
    DT.fit(X_train,Y_train)
    DT_predictions = DT.predict(X_test)
    
    #for k-NN classifier with neighbors = 1 
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, Y_train) 
    knn_predictions = knn.predict(X_test) 
    
    #providing columns with headers labels in google spreadsheet
    cell_list = worksheet.update_cell(1,1, 'RF- F1')
    cell_list = worksheet.update_cell(1,2, 'RF- ACC')
    cell_list = worksheet.update_cell(1,3, 'RBF- F1')
    cell_list = worksheet.update_cell(1,4, 'RBF- ACC')
    cell_list = worksheet.update_cell(1,5, 'DT- F1')
    cell_list = worksheet.update_cell(1,6, 'DT- ACC')
    cell_list = worksheet.update_cell(1,7, 'kNN- F1')
    cell_list = worksheet.update_cell(1,8, 'kNN- ACC')
    
    #print the output
    #update the google sheet with the outputs
    #for RF
    print('RF F1score:', f1_score(Y_test, clf_predict, average= 'weighted'))
    cell_list = worksheet.update_cell(2, 1, f1_score(Y_test, clf_predict, average= 'weighted'))
    print('RF Accuracy:',accuracy_score(Y_test, clf_predict))
    cell_list = worksheet.update_cell(2, 2, accuracy_score(Y_test, clf_predict))
    
    #for RBF
    print('RBF F1score:', f1_score(Y_test, svc_predictions, average= 'weighted'))
    cell_list = worksheet.update_cell(2, 3, f1_score(Y_test, svc_predictions, average= 'weighted'))                                  
    print('RBF Accuracy:', accuracy_score(Y_test, svc_predictions))
    cell_list = worksheet.update_cell(2, 4, accuracy_score(Y_test, svc_predictions)) 
    
    #uncomment to check f1 score of MLP
    #results for MLP have not been linked to google spreadsheet
    #print('MLP F1score:', f1_score(Y_test, mlp_predictions, average= 'weighted'))
    #uncomment to check accuracy of MLP
    #results for MLP have not been linked to google spreadsheet                               
    #print('MLP Accuracy:', accuracy_score(Y_test, mlp_predictions))

    #for DT
    print('DT F1score:', accuracy_score(Y_test, DT_predictions))
    cell_list = worksheet.update_cell(2, 5, f1_score(Y_test, DT_predictions, average= 'weighted'))
    print('DT Accuracy:', accuracy_score(Y_test, DT_predictions))
    cell_list = worksheet.update_cell(2, 6, accuracy_score(Y_test, DT_predictions))                                      
    
    #for k-NN
    print('kNN F1score:', f1_score(Y_test, knn_predictions, average= 'weighted'))
    cell_list = worksheet.update_cell(2, 7, f1_score(Y_test, knn_predictions, average= 'weighted'))                                  
    print('kNN F1score:', accuracy_score(Y_test, knn_predictions))
    cell_list = worksheet.update_cell(2, 8, accuracy_score(Y_test, knn_predictions))
    print("\n")

for CV value = 1 we have the following output:

RF F1score: 0.9982459370770531
RF Accuracy: 0.99825
RBF F1score: 0.9671318684688324
RBF Accuracy: 0.959
DT F1score: 0.99825
DT Accuracy: 0.99825
kNN F1score: 1.0
kNN F1score: 1.0


for CV value = 2 we have the following output:

RF F1score: 0.9983751197212323
RF Accuracy: 0.998375
RBF F1score: 0.9701925581793346
RBF Accuracy: 0.962625
DT F1score: 0.998375
DT Accuracy: 0.998375
kNN F1score: 1.0
kNN F1score: 1.0


for CV value = 3 we have the following output:

RF F1score: 0.9978772329521425
RF Accuracy: 0.997875
RBF F1score: 0.9687003736127294
RBF Accuracy: 0.9615
DT F1score: 0.997875
DT Accuracy: 0.997875
kNN F1score: 0.999501023292927
kNN F1score: 0.9995


for CV value = 4 we have the following output:

RF F1score: 0.9975124713577868
RF Accuracy: 0.9975
RBF F1score: 0.9672864768845275
RBF Accuracy: 0.95875
DT F1score: 0.9975
DT Accuracy: 0.9975
kNN F1score: 0.9993750775693027
kNN F1score: 0.999375


for CV value = 5 we have the following